In [1]:
from spacy.en import English
nlp = English()

In [30]:
import time as tm
import numpy as np
import pandas as pd
import re

verbose = False
np.random.seed(14)
input_csv = "data/snippets.csv"
full = pd.read_csv(input_csv) # , header=None)
print full.shape

full = full.dropna(axis=0)
print full.shape

full = full[full['Content'].str.len() < 600]
print full.shape

full = full[0:10000]
print full.shape

(65499, 3)
(64458, 3)
(64434, 3)
(10000, 3)


In [31]:
preX = full['Content']
preX = preX.apply(lambda row: row.strip())
preX = preX.apply(lambda row: row.replace('-', ' '))
parsed = preX.apply(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
reindexed = parsed.reset_index(drop=True)
# X = reindexed

In [32]:
postX = []
for idx, sentence in enumerate(preX):
    found_entities = []
    for ent in reindexed[idx].ents:
        if ent.label_ == 'ORG':
            found_entities.append( (ent.text.encode('utf-8'), 'ORG') )
        if ent.label_ == 'PERSON':
            found_entities.append( (ent.text.encode('utf-8'), 'PERSON') )
#         if ent.label_ == 'MONEY':
#             found_entities.append( (ent.text.encode('utf-8'), 'MONEY') )
    for word in found_entities:
        sentence = sentence.replace(word[0], word[1])
    postX.append(sentence)
    if (idx%2500 == 0):
        print idx

postX = pd.Series(postX)
parsed = postX.map(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
tokenized = parsed.map(lambda row: [token.orth_ for token in row])
X = tokenized

print len(X)

0
2500
5000
7500
10000


In [76]:
y = full['Tags']

y = y.str.split(';').str[0]
# print y.value_counts()

In [78]:
y = y.str.replace('total-', '')
y = y.str.replace('filing-', '')
y = y.str.replace('key_person_', '')

y = y.str.replace('lawsuit', 'OTHER')
y = y.str.replace('office_move', 'OTHER')
y = y.str.replace('geo', 'OTHER')
y = y.str.replace('reorg', 'OTHER')
y = y.str.replace('patent', 'OTHER')
y = y.str.replace('conference-sponsorship', 'OTHER')
y = y.str.replace('executive-promotion', 'OTHER')
y = y.str.replace('bankruptcy', 'OTHER')
y = y.str.replace('event-participant', 'OTHER')
y = y.str.replace('scale', 'OTHER')
y = y.str.replace('team-grew', 'OTHER')
y = y.str.replace('executive-publication', 'OTHER')
y = y.str.replace('job-posting', 'OTHER')
y = y.str.replace('leadership-change', 'OTHER')

y = y.fillna('OTHER')
jake = y.value_counts()
print len(jake)
print jake

21
funding        1511
strategy       1502
hire            825
partnership     818
product         721
acquisition     668
revenue         662
OTHER           468
overview        447
users           425
headcount       375
award           368
valuation       198
turmoil         170
founding        167
customers       156
market          154
departure       131
sector          101
ipo              76
competitors      57
dtype: int64


In [41]:
y = y.reset_index(drop=True)
y = y[0:10000]

In [33]:
max = 0
for exp in X:
    if (len(exp) > max):
        max = len(exp)
print max

100


In [44]:
len(y)


10000

In [45]:
print y.shape
y.name="Tags"
print X.shape
X.name="Content"
data = pd.concat([X, y], axis=1)
print data.shape

(10000,)
(10000,)
(10000, 2)


In [59]:
data['Tokens'] = data.apply(lambda row: len(row["Content"]), axis=1)
print data.head()
print len(data)
# print type(buck10)
# print len(buck10)
# print buck10.head()

                                             Content         Tags  Tokens
0  [PERSON, ,, creators, of, the, first, guided, ...      funding      36
1  [ORG, ,, the, new, venture, by, ORG, founders,...     overview      23
2  [The, firm, will, launch, a, beta, program, in...      product      30
3  [ORG, has, added, task, management, to, its, s...      product      22
4  [But, with, this, latest, release, ,, the, goa...  competitors      32
10000


In [65]:
buck1 = data[ data["Tokens"]<=10 ]
buck2 = data[ (data["Tokens"]<=20) & (data["Tokens"]>10) ]
buck3 = data[ (data["Tokens"]<=30) & (data["Tokens"]>20) ]
buck4 = data[ (data["Tokens"]<=40) & (data["Tokens"]>30) ]
buck5 = data[ (data["Tokens"]<=50) & (data["Tokens"]>40) ]
buck6 = data[ (data["Tokens"]<=60) & (data["Tokens"]>50) ]
buck7 = data[ (data["Tokens"]<=80) & (data["Tokens"]>60) ]
buck8 = data[ data["Tokens"]>80 ]

In [74]:
myBuks = [buck10, buck20, buck30, buck40, buck50, buck60, buck80, buckmax]
totalRows = 0

for bucket in myBuks:
    sep = len(bucket)
    print sep
    totalRows += sep

print totalRows
print len(data)

128
1774
3335
2708
1370
460
197
28
10000
10000


In [ ]:
# print data['Content'][8771]
# print data['Content'][10731]
# print data['Content'][3451]
#  Tricky token parsing: [10731, 37877, 8771, 3451, 42795]

In [73]:
type(buck50)

pandas.core.frame.DataFrame

In [75]:
buck10.to_pickle('data/buck1.p')
buck20.to_pickle('data/buck2.p')
buck30.to_pickle('data/buck3.p')
buck40.to_pickle('data/buck4.p')
buck50.to_pickle('data/buck5.p')
buck60.to_pickle('data/buck6.p')
buck80.to_pickle('data/buck7.p')
buckmax.to_pickle('data/buck8.p')